<a href="https://colab.research.google.com/github/SilverSurferClash/Sales_analysis/blob/main/Import_clean_SFDC_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!pip freeze > requirements.txt

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import os
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 2)
#Use option to round floats to two decimals
#pd.set_option('display.float_format',  '{:,.2f}'.format)
# Use option to display $ dollars
#pd.set_option('display.float_format', '${:.2f}'.format)

# Plotting pretty figures and avoid blurry images
#%config InlineBackend.figure_format = 'retina'
# Larger scale for plots in notebooks
#sns.set_context('notebook')


# Enable multiple cell outputs
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore')              


In [4]:
os.getcwd()

'/content'

In [5]:
os.chdir("/content/drive/MyDrive/Colab_Notebooks/Sales_Analysis")

Import helper data

In [8]:
#Import lookup table for countries
lookup_countries = pd.read_excel("./lookup.xlsx", sheet_name = 0)
#Import the data from SFDC as a excel file
import_sfdc = pd.read_excel("./All Data EUAA-2022-10-25-12-29-00.xlsx", sheet_name = 0, 
                            skiprows = 10)
#Create dictionairy for maping the lookup value
country_dict = dict(zip(lookup_countries["Reporting_Country_short"], lookup_countries["Account_Reporting_Country"]))
#Import lookup table for countries
lookup_type = pd.read_excel("./lookup.xlsx", sheet_name = "Type")
#Create dictionairy for maping the lookup values for LIMS ID
type_dict = dict(zip(lookup_type["LIMS Project ID"], lookup_type["Type"]))


In [9]:
#Identify which LIMS id are not in the lookup
unique_LIMS_id = list(set(import_sfdc["LIMS Project ID"]))
unique_lookup_id = list(set(lookup_type["LIMS Project ID"]))
list(set(unique_LIMS_id) - set(unique_lookup_id ))

[nan, 'AZN_test_96', 'OCT_1R', 'GAP_04', 'JNS_01', 'RHI_04', 'GAP_09']

#### Import and clean up the data from SFDC

In [10]:

#Remove the first two columns
import_sfdc = import_sfdc.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1)
#Drop the last five rows
import_sfdc = import_sfdc.iloc[0:-5]
#Cast Date Received to datetime object 
import_sfdc["Date Received"] = pd.to_datetime(import_sfdc["Date Received"])
#Replace the two letter country code with the full country name
import_sfdc["Reporting Country"] = import_sfdc["Reporting Country"].map(country_dict)

#import_sfdc.tail(6)
import_sfdc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14977 entries, 0 to 14976
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Accession #            14977 non-null  object        
 1   Reporting Country      14977 non-null  object        
 2   Billing Type           14949 non-null  object        
 3   LIMS Project ID        14977 non-null  object        
 4   Account: Account Name  14977 non-null  object        
 5   Contact: Full Name     14977 non-null  object        
 6   Date Received          14977 non-null  datetime64[ns]
 7   Final Report Date      13479 non-null  object        
 8   Test Type              14977 non-null  object        
 9   Cancer Type            14538 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 1.1+ MB


In [11]:
import_sfdc.sample(1)

,Accession #,Reporting Country,Billing Type,LIMS Project ID,Account: Account Name,Contact: Full Name,Date Received,Final Report Date,Test Type,Cancer Type
6110,A0309999,UNITED KINGDOM,Self-Pay Sponsored,GHI_01,The Royal Marsden NHS Foundation Trust,Ian Smith,2020-12-05,12/12/2020,Guardant 360,Breast Carcinoma


#### Explore the data

In [12]:
import_sfdc["Reporting Country"].value_counts()

SPAIN              4759
UNITED KINGDOM     3360
FRANCE             1367
ITALY              1210
CANADA              853
TURKEY              694
GERMANY             577
BRAZIL              479
POLAND              287
BELGIUM             268
HUNGARY             153
AUSTRIA             142
SWITZERLAND         127
MEXICO              112
IRELAND             110
CHILE                74
ARGENTINA            66
FINLAND              55
BELARUS              52
PORTUGAL             48
GREECE               39
UKRAINE              35
NETHERLANDS          33
DENMARK              13
PERU                 13
LITHUANIA            11
CZECH REPUBLIC       10
SWEDEN                9
SLOVENIA              7
ROMANIA               5
COLOMBIA              4
NORTH MACEDONIA       2
CROATIA               1
LUXEMBOURG            1
COSTA RICA            1
Name: Reporting Country, dtype: int64

In [20]:
import_sfdc["Type"] = import_sfdc["LIMS Project ID"].map(type_dict)

In [19]:
import_sfdc.Type.value_counts()

Biopharma                  6305
Commercial                 3859
Lunar                      1801
GAP                        1778
GH Trials (Prospective)    1130
Name: Type, dtype: int64

In [21]:
import_sfdc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14977 entries, 0 to 14976
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Accession #            14977 non-null  object        
 1   Reporting Country      14977 non-null  object        
 2   Billing Type           14949 non-null  object        
 3   LIMS Project ID        14977 non-null  object        
 4   Account: Account Name  14977 non-null  object        
 5   Contact: Full Name     14977 non-null  object        
 6   Date Received          14977 non-null  datetime64[ns]
 7   Final Report Date      13479 non-null  object        
 8   Test Type              14977 non-null  object        
 9   Cancer Type            14538 non-null  object        
 10  Type                   14873 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 1.3+ MB


In [14]:
import_sfdc["Billing Type"].value_counts()

Pharma : ROW                              5631
Pharma : US                               2370
Self-Pay Patient                          1687
Distributors : ROW                        1610
Do Not Bill - LUNAR                       1199
Do Not Bill - Cancelled                    720
Clinical Trial : ROW                       685
Self-Pay Sponsored                         425
Research : ROW                             420
Do Not Bill - ROW                          160
Other                                       11
Do Not Bill - Promotional                   10
Private Insurance/Employee Health Plan       8
Do Not Bill - Medical Affairs                7
Clinical Trial : US                          5
Research : US                                1
Name: Billing Type, dtype: int64

In [22]:
import_sfdc["Test Type"].value_counts()

Guardant 360            12793
LUNAR1_CRC                766
LUNAR2                    643
LUNAR1_3                  402
Guardant360 CDx EU        371
Guardant360 Response        1
Guardant360 CDx             1
Name: Test Type, dtype: int64